In [ ]:
# ===============================================================
# 03_kerr_flux_sim.ipynb  — Kerr-enhanced quaternionic flux (final fix)
# ===============================================================
!pip install --quiet scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from math import sqrt, pi

# ---------- 物理常數 (SI) ----------------------------------------
G, c = 6.67430e-11, 2.99792458e8
M_sun, M_pl = 1.98847e30, 2.176434e-8   # kg

# ---------- 黑洞參數 (Sgr A*) ------------------------------------
M_bh   = 4.0e6 * M_sun                  # 4×10⁶ M⊙
a_star = 0.5                            # 自旋參數 0 ≤ a* < 1

r_g_m  = 2*G*M_bh/c**2                  # Schwarzschild 半徑 (m)
print(f"r_g = {r_g_m/1e3:.2e} km,  a* = {a_star}")

# ---------- 無因次化： r̂ = r/r_g ---------------------------------
a_hat  = a_star
Mratio = M_bh / M_pl                    # ≈3.65×10⁴⁴

Δ  = lambda x: x**2 - x + a_hat**2
dΔ = lambda x: 2*x - 1

# ---------- curvature-driven coupling λ̃( r̂ ) --------------------
κ = 2.0e-2
λ_tilde = lambda x: 2*κ*Mratio / x**3   # 論文 Eq.(E.4)

# ---------- ODE 定義 ---------------------------------------------
def rhs(x, y):
    ψ, χ = y
    src  = x**2 * λ_tilde(x) * ψ*(ψ**2 - 1)
    dxd  = (2*x*Δ(x) + x**2*dΔ(x))
    χp   = (src - dxd*χ) / (x**2 * Δ(x))
    return [χ, χp]

# ---------- 積分範圍：從 r̂=2 → 100（避開視界附近剛性區） -----------
x_start = 2.0
x_end   = 100.0
x_eval  = np.geomspace(x_start, x_end, 800)

# 初始條件：ψ(2)=小非零；ψ'(2)=0
y0 = [1e-6, 0.0]

sol = solve_ivp(rhs,
                t_span=(x_start, x_end),
                y0=y0,
                t_eval=x_eval,
                method="Radau",
                rtol=1e-7,
                atol=1e-10,
                max_step=1.0)

if not sol.success:
    raise RuntimeError(sol.message)

ψ_arr  = sol.y[0]
ε_arr  = 2*ψ_arr

# ---------- 論文 analytic ansatz 對照 ----------------------------
r_s0_hat = (8.5e3 * 3.085677e16) / r_g_m   # 8.5 kpc → 無因次
boost    = 1.0 / (1 - a_hat/x_eval)
ε_ansatz = 2*np.tanh(x_eval / (r_s0_hat * boost))

# ------------------------- 繪圖 -----------------------------------
plt.figure(figsize=(7,5))
plt.loglog(x_eval,   ε_arr,    lw=2, label="Numerical ε(r)")
plt.loglog(x_eval, ε_ansatz, '--', lw=2, label="Analytic ansatz")
plt.axvline(2.0,     ls=":", c='k', alpha=.5)     # 標示起點
plt.text(2.1, 1e-8, "start", color='k')
plt.xlabel(r"$\hat r = r/r_g$")
plt.ylabel(r"$\epsilon(r)$")
plt.title("Quaternionic flux in Kerr spacetime (fixed & shifted start)")
plt.legend(); plt.grid(ls="--", alpha=0.3); plt.tight_layout()
plt.show()

# ---------- δc_g/c 估算 (LISA, f=0.01Hz) -------------------------
f_LISA = 1e-2                       # Hz
k_SI   = 2*pi*f_LISA / c            # s⁻¹
k_hat  = k_SI * r_g_m / c           # 無因次
idx10  = np.argmin(np.abs(x_eval - 10))
ψ10    = ψ_arr[idx10]
deltac = (ψ10**2 * (a_hat/x_eval[idx10])**2) / (2 * k_hat**2)

print(f"\n在 10 r_g： ε ≈ {ε_arr[idx10]:.2e}")
print(f"預測 δc_g/c ≈ {deltac:.2e}")
